## Simple Neural Network with MNIST dataset

In [ ]:
import tensorflow as tf
import numpy as np
import math
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images,mnist.test.labels

In [ ]:
class FCNN(object):
    
    def __init__(self, sizes, X, Y, epoches=5, applyPCA=False):
        # Hyperparameters
        self._sizes = sizes
        self._X = X
        self._Y = Y
        self._learning_rate =  1.0
        self._momentum = 0.0
        self._epoches = epoches
        self._batchsize = 100
        self.input_size = X.shape[1]
        self.output_size = Y.shape[1]
        self.w_tf_list = []
        self.b_tf_list = []
        
        # Weights and Biases
        tf.reset_default_graph() # reset tensorflow graph
        with tf.variable_scope("FC"):            
            pre_size = self.input_size
            for i, size in enumerate(self._sizes + [Y.shape[1]]):
                self.w_tf_list.append(tf.get_variable(str('w'+str(i+1)), shape=[pre_size, size], dtype=tf.float32))
                # initialize biases
                self.b_tf_list.append(tf.get_variable(str('b'+str(i+1)), shape=[size], dtype=tf.float32))
                pre_size = size
    
    def train(self):
        # Placeholders
        Xph = tf.placeholder("float", [None, self.input_size])
        Yph = tf.placeholder("float", [None, self.output_size])
        
        ##LAYERs
        layers_list = []
        pre_layer_out = Xph
        for i, size in enumerate(self._sizes):
            layers_list.append(tf.nn.sigmoid(tf.matmul(pre_layer_out, self.w_tf_list[i]) + self.b_tf_list[i]))
            pre_layer_out = layers_list[-1]
        # Output Layer
        Y_hat = tf.nn.softmax(tf.matmul(layers_list[-1], self.w_tf_list[-1]) + self.b_tf_list[-1])
        
        # Loss
        cross_entropy = tf.reduce_mean(-tf.reduce_sum(Yph * tf.log(Y_hat), reduction_indices=[1]))
        
        # accuracy
        correct_prediction = tf.equal(tf.argmax(Y_hat, 1), tf.argmax(Yph, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        # training
        train_step = tf.train.GradientDescentOptimizer(self._learning_rate).minimize(cross_entropy)
        
        #some GPU configuration
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
        config = tf.ConfigProto(gpu_options=gpu_options)
        config.gpu_options.allow_growth=True
        with tf.Session(config=config) as sess:
            sess.run(tf.global_variables_initializer())
            pre_acc = 0
            for step in range(self._epoches):
                #For each step
                for start, end in zip(range(0, len(self._X), self._batchsize), range(self._batchsize, len(self._X), self._batchsize)):
                    
                    sess.run(train_step, feed_dict={Xph: self._X[start:end], Yph: self._Y[start:end]})
                print("Accuracy at epoch {}: Train={:.4f}, Test={:.4f}"
                      .format(step+1, 
                              sess.run(accuracy, feed_dict={Xph: mnist.train.images, Yph: mnist.train.labels}), 
                              sess.run(accuracy, feed_dict={Xph: mnist.test.images, Yph: mnist.test.labels})))

In [ ]:
tf.set_random_seed(1)
layers_size = [500, 200, 50]
nNet = FCNN(layers_size, trX, trY, epoches=70)
nNet.train()